### Modelo de Otimização Linear para Sudoku


In [3]:
from amplpy import AMPL, ampl_notebook

In [14]:
ampl_notebook(modules=["highs"], license_uuid="2e003210-ce0e-4ca9-bb4e-5d4a704b43a3")

# Inicializa o ambiente AMPL
ampl = AMPL()

# Define o modelo AMPL
modelo = """
var x {i in 1..9, j in 1..9, k in 1..9} >=0 <=1;

minimize cte: 81;

subject to ApenasUmaAparicao {i in 1..9, j in 1..9}:
    sum{k in 1..9} x[i,j,k] = 1;

subject to Linha {i in 1..9, k in 1..9}:
    sum{j in 1..9} x[i,j,k] = 1;

subject to Coluna {j in 1..9, k in 1..9}:
    sum{i in 1..9} x[i,j,k] = 1;

subject to Subgrade {s in 0..2, t in 0..2, k in 1..9}:
    sum{i in 3*s+1..3*s+3, j in 3*t+1..3*t+3} x[i,j,k] = 1;

subject to preenche139: x[1,3,9] = 1; 
subject to preenche147: x[1,4,7] = 1; 
subject to preenche249: x[2,4,9] = 1; 
subject to preenche343: x[3,4,3] = 1; 
subject to preenche193: x[1,9,3] = 1; 
subject to preenche271: x[2,7,1] = 1; 
subject to preenche366: x[3,6,6] = 1; 
subject to preenche398: x[3,9,8] = 1; 
subject to preenche419: x[4,1,9] = 1; 
subject to preenche436: x[4,3,6] = 1; 
subject to preenche454: x[4,5,4] = 1; 
subject to preenche512: x[5,1,2] = 1; 
subject to preenche533: x[5,3,3] = 1; 
subject to preenche565: x[5,6,5] = 1; 
subject to preenche596: x[5,9,6] = 1; 
subject to preenche685: x[6,8,5] = 1; 
subject to preenche697: x[6,9,7] = 1; 
subject to preenche723: x[7,2,3] = 1; 
subject to preenche762: x[7,6,2] = 1; 
subject to preenche788: x[7,8,8] = 1; 
subject to preenche795: x[7,9,5] = 1; 
subject to preenche818: x[8,1,8] = 1; 
subject to preenche911: x[9,1,1] = 1; 

"""

# Carrega modelo e dados
ampl.eval(modelo)

ampl.set_option('solver', 'highs')

# Resolve o problema
ampl.solve()

# Recupera a solução e formata o Sudoku
solution = ampl.get_variable("x").to_dict()  # Dicionário com os valores de x[i,j,k]

# Inicializa uma matriz 9x9 com zeros
sudoku_grid = [[0 for _ in range(9)] for _ in range(9)]

# Preenche a matriz com os valores encontrados
for (i, j, k), val in solution.items():
    if val > 0.5:  # Se x[i,j,k] ≈ 1 (binário)
        sudoku_grid[i-1][j-1] = k  # Índices em Python são 0-based

# Imprime o Sudoku formatado
print("\nSolução do Sudoku:")
for i in range(9):
    if i % 3 == 0 and i != 0:
        print("-" * 21)  # Linha divisória entre blocos 3x3
    row = []
    for j in range(9):
        if j % 3 == 0 and j != 0:
            row.append("|")  # Separador vertical entre blocos 3x3
        row.append(str(sudoku_grid[i][j]))
    print(" ".join(row))

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 81
0 simplex iterations
0 barrier iterations

Solução do Sudoku:
6 2 9 | 7 8 1 | 5 4 3
3 8 7 | 9 5 4 | 1 6 2
5 4 1 | 3 2 6 | 9 7 8
---------------------
9 5 6 | 2 4 7 | 8 3 1
2 7 3 | 8 1 5 | 4 9 6
4 1 8 | 6 3 9 | 2 5 7
---------------------
7 3 4 | 1 9 2 | 6 8 5
8 9 2 | 5 6 3 | 7 1 4
1 6 5 | 4 7 8 | 3 2 9


Exercício 12.19 do livro Williams, H. P. (2013). Model building in mathematical programming. John Wiley & Sons.

In [15]:
from pulp import *

# Cria o problema de minimização
prob = LpProblem("Problema_de_Distribuicao_12.19", LpMinimize)

# Variáveis de decisão
# Fábricas para Depósitos
x_LN = LpVariable("Liverpool_to_Newcastle", 0, None, LpContinuous)
x_LB = LpVariable("Liverpool_to_Birmingham", 0, None, LpContinuous)
x_LL = LpVariable("Liverpool_to_London", 0, None, LpContinuous)
x_LE = LpVariable("Liverpool_to_Exeter", 0, None, LpContinuous)
x_BB = LpVariable("Brighton_to_Birmingham", 0, None, LpContinuous)
x_BL = LpVariable("Brighton_to_London", 0, None, LpContinuous)
x_BE = LpVariable("Brighton_to_Exeter", 0, None, LpContinuous)

# Fábricas para Clientes
x_LC1 = LpVariable("Liverpool_to_C1", 0, None, LpContinuous)
x_LC3 = LpVariable("Liverpool_to_C3", 0, None, LpContinuous)
x_LC4 = LpVariable("Liverpool_to_C4", 0, None, LpContinuous)
x_LC6 = LpVariable("Liverpool_to_C6", 0, None, LpContinuous)
x_BC1 = LpVariable("Brighton_to_C1", 0, None, LpContinuous)

# Depósitos para Clientes
x_NC1 = LpVariable("Newcastle_to_C1", 0, None, LpContinuous)
x_NC2 = LpVariable("Newcastle_to_C2", 0, None, LpContinuous)
x_NC3 = LpVariable("Newcastle_to_C3", 0, None, LpContinuous)
x_NC4 = LpVariable("Newcastle_to_C4", 0, None, LpContinuous)
x_NC6 = LpVariable("Newcastle_to_C6", 0, None, LpContinuous)
x_BC2 = LpVariable("Birmingham_to_C2", 0, None, LpContinuous)
x_BC3 = LpVariable("Birmingham_to_C3", 0, None, LpContinuous)
x_BC4 = LpVariable("Birmingham_to_C4", 0, None, LpContinuous)
x_BC5 = LpVariable("Birmingham_to_C5", 0, None, LpContinuous)
x_LC2 = LpVariable("London_to_C2", 0, None, LpContinuous)
x_LC3 = LpVariable("London_to_C3", 0, None, LpContinuous)
x_LC5 = LpVariable("London_to_C5", 0, None, LpContinuous)
x_LC6 = LpVariable("London_to_C6", 0, None, LpContinuous)
x_EC3 = LpVariable("Exeter_to_C3", 0, None, LpContinuous)
x_EC4 = LpVariable("Exeter_to_C4", 0, None, LpContinuous)
x_EC5 = LpVariable("Exeter_to_C5", 0, None, LpContinuous)
x_EC6 = LpVariable("Exeter_to_C6", 0, None, LpContinuous)

# Função Objetivo: Minimizar o custo total
prob += (0.5*x_LN + 0.5*x_LB + 1.0*x_LL + 0.2*x_LE + 
         0.3*x_BB + 0.5*x_BL + 0.2*x_BE +
         1.0*x_LC1 + 1.5*x_LC3 + 2.0*x_LC4 + 1.0*x_LC6 + 
         2.0*x_BC1 +
         1.5*x_NC2 + 0.5*x_NC3 + 1.5*x_NC4 + 1.0*x_NC6 +
         1.0*x_BC1 + 0.5*x_BC2 + 0.5*x_BC3 + 1.0*x_BC4 + 0.5*x_BC5 +
         1.5*x_LC2 + 2.0*x_LC3 + 0.5*x_LC5 + 1.5*x_LC6 +
         0.2*x_EC3 + 1.5*x_EC4 + 0.5*x_EC5 + 1.5*x_EC6), "Custo_Total"

# Restrições de Capacidade das Fábricas (em milhares de toneladas)
prob += x_LN + x_LB + x_LL + x_LE + x_LC1 + x_LC3 + x_LC4 + x_LC6 <= 150, "Capacidade_Liverpool"
prob += x_BB + x_BL + x_BE + x_BC1 <= 200, "Capacidade_Brighton"

# Restrições de Capacidade dos Depósitos
prob += x_LN + x_NC1 + x_NC2 + x_NC3 + x_NC4 + x_NC6 <= 70, "Capacidade_Newcastle"
prob += x_LB + x_BB + x_BC2 + x_BC3 + x_BC4 + x_BC5 <= 50, "Capacidade_Birmingham"
prob += x_LL + x_BL + x_LC2 + x_LC3 + x_LC5 + x_LC6 <= 100, "Capacidade_London"
prob += x_LE + x_BE + x_EC3 + x_EC4 + x_EC5 + x_EC6 <= 40, "Capacidade_Exeter"

# Restrições de Demanda dos Clientes
prob += x_LC1 + x_BC1 + x_NC1 == 50, "Demanda_C1"
prob += x_NC2 + x_BC2 + x_LC2 == 10, "Demanda_C2"
prob += x_LC3 + x_NC3 + x_BC3 + x_LC3 + x_EC3 == 40, "Demanda_C3"
prob += x_LC4 + x_NC4 + x_BC4 + x_EC4 == 35, "Demanda_C4"
prob += x_BC5 + x_LC5 + x_EC5 == 60, "Demanda_C5"
prob += x_LC6 + x_NC6 + x_LC6 + x_EC6 == 20, "Demanda_C6"

# Restrições de Fornecimento Direto (impossibilidades)
# Já implementadas pela não criação das variáveis impossíveis

# Resolve o problema
prob.solve()

# Imprime o status da solução
print("Status:", LpStatus[prob.status])

# Imprime o valor ótimo da função objetivo
print("Custo Total Mínimo = £", value(prob.objective)/1000, "mil")

# Imprime as variáveis com valores positivos na solução ótima
print("\nSolução Ótima:")
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue, "mil toneladas")

Status: Optimal
Custo Total Mínimo = £ 0.098 mil

Solução Ótima:
Birmingham_to_C2 = 10.0 mil toneladas
Birmingham_to_C4 = 35.0 mil toneladas
Birmingham_to_C5 = 5.0 mil toneladas
Exeter_to_C3 = 40.0 mil toneladas
London_to_C5 = 55.0 mil toneladas
Newcastle_to_C1 = 50.0 mil toneladas
Newcastle_to_C6 = 20.0 mil toneladas
